### Install dependencies

In [1]:
# same deal for gdrive and kaggle
from google.colab import drive
drive.mount('/content/drive')

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/.kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install -q kaggle


Mounted at /content/drive
rm: cannot remove '/root/.kaggle': No such file or directory


In [2]:
# install dataset and unzip

!rm -r dataset

!kaggle datasets download -d saumitgp/occupancy-detection-dataset
!mkdir dataset
!unzip occupancy-detection-dataset.zip -d dataset

rm: cannot remove 'dataset': No such file or directory
  0% 0.00/213k [00:00<?, ?B/s]
100% 213k/213k [00:00<00:00, 54.1MB/s]
Archive:  occupancy-detection-dataset.zip
  inflating: dataset/OccupancyData/DataTest.csv  
  inflating: dataset/OccupancyData/DataTraining.csv  


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# the balance is weird, I will merge the train and test files first and split them to 8:2
df = pd.concat([
        pd.read_csv('dataset/OccupancyData/DataTraining.csv',index_col=[0]),
        pd.read_csv('dataset/OccupancyData/DataTest.csv',index_col=[0])
    ])
raw_vals = df.drop(columns=['date']).values
X,y = raw_vals[:,:-1],raw_vals[:,-1]
# pad X with 1 in the beginning, this will serve as our bias term
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=0)

# decision trees don't need scaling! :)

### Task 1

Implement a Decision Tree Classifier for your classification problem. You may use a built-in package to implement your classifier. Try modifying one or more of the input parameters and describe what changes you notice in your results. Clearly describe how these factors are affecting your output.

In [8]:
from sklearn.tree import DecisionTreeClassifier


results = []
# maximum depth of the tree
for d in [2,3,4,5,6]:
    for l in [1,.2,.1]:
    # ccp_alpha, this is basically the complexity threshold
        for alpha in [0,.1,.3]:
            params = {
                'random_state': 0,
                'max_depth': d,
                'ccp_alpha': alpha,
                'min_samples_leaf':l
            }
            tree = DecisionTreeClassifier(**params)

            tree.fit(X_train,y_train)

            results.append({
                **params,
                'test_score': tree.score(X_test,y_test),
                'train_score':tree.score(X_train,y_train),
            })

results = pd.DataFrame(results)

results

,random_state,max_depth,ccp_alpha,min_samples_leaf,test_score,train_score
0,0,2,0.0,1.0,0.987427,0.992526
1,0,2,0.1,1.0,0.987147,0.991897
2,0,2,0.3,1.0,0.987147,0.991897
3,0,2,0.0,0.2,0.987147,0.991897
4,0,2,0.1,0.2,0.987147,0.991897
5,0,2,0.3,0.2,0.987147,0.991897
6,0,2,0.0,0.1,0.987147,0.991897
7,0,2,0.1,0.1,0.987147,0.991897
8,0,2,0.3,0.1,0.987147,0.991897
9,0,3,0.0,1.0,0.987427,0.992596


### Task 2

From the Bagging and Boosting ensemble methods pick any one algorithm 
from each category. Implement both the algorithms using the same data. Use k-fold cross 
validation to find the effectiveness of both the models. Comment on the difference/similarity of 
the results.

### Task 3

Compare the effectiveness of the three models implemented above. Clearly 
describe the metric you are using for comparison. Describe (with examples) Why is this 
metric(metrics) suited/appropriate for the problem at hand? How would a choice of a different 
metric impact your results? Can you demonstrate that